# Sentiment analysis of Dutch COVID-19 related twitter messages

## Research question
*Is it possible to predict whether a COVID-19 related Twitter message is positve or negative?*

#### Background information about the project
* This project is part of the Ironhack Data Analytics course I followed between June and December 2020.
* Project duration: 14th - 28th November 2020.
* Overarching goals:
   1. Learn to learn and apply new tools and methodologies. Keeping up with developments and teach yourself new tools is an important trait of a data analyst;
   2. Focus on learning a skill transferrable to the final project.
* Specific goals for Natural Language Processing projects:
    * Design and build an NLP prototype. 
    * Integrate a machine learning algorithm in an NLP framework (e.g. a chatbot or sentiment analyzer)  
    * Use evaluation metrics to ground your claims and test your model. 
    * Communicate the results of your analysis clearly, accurately, and engagingly. Demonstrate your working prototype. 
    * Learn to tailor the style of your communication to the audience.

#### Download the dataset
Please download the dataset from [here](https://drive.google.com/file/d/1JnvJrmTC0XgA5XwU9Tywxb8Cliqe8Gi8/view?usp=sharing) and place it in the data folder.

#### Context about the data
The dataset contains around 17.000 Dutch twitter messages. The Twitter messages were obtained via the Twitter API using Tweepy. You can find the code in Python file 'How to get data from Twitter' in this folder.

The Twitter messages have the following characteristics:
- Dutch language label.
- Created between 17 Nov 2020 and 26 Nov 2020.
- Retweets are filtered out of the dataset.
- Each message has at least one of the following hashtags: #corona, #CoronaNL, #coronanederland, #coronavirus, #coronavirusnl, #coronavirusNederland, #covid19, #COVID19NL, #covid19nederland, #covid-19, #covid_19, #covid_19nl and #Covid_19Nederland.

The first data cleaning was already done obtaining the messages. Only two colums are kept and can be found in this dataset: the creation date and text of the message.

#### Import libraries
First, install and import the necessary libraries to conduct the analysis.

In [1]:
import pandas as pd
import numpy as np
import random
import re
import emoji
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob_nl import PatternTagger, PatternAnalyzer

# Install libraries and download packages if needed:
# pip install emoji
# pip install textblob
# pip install textblob-nl
# pip install tensorflow
# pip install keras
# nltk.download('stopwords')

#### Import datasets
Secondly, import the dataset.

In [2]:
tweets = pd.read_csv('data\\twitterdata_NEW.csv')

#### Inspecting & cleaning the data
1. We first set the maximum column width to infinite number of characters. The default is 50, which means we cannot read the whole message.
2. We print the first 5 rows of the dataframe to see what the dataset looks like.
3. The datatype of both columns is 'object'. Since 'created at' is a date, we convert this column to datetime.
4. At midnight on Thursday the 19th of November, part of the lockdown measures in the Netherlands was widened. Meaning people were allowed to have more vistors at home and visiting a museum for instance. This might influence the sentiment of the messages. Therefore we add an extra column to the dataframe: the time difference between the widening of the measures and creation of the message.

In [3]:
# Set column width to infinite number of characters
pd.set_option('max_colwidth', None)

In [4]:
# Inspect the first 5 rows
tweets.head()

,created_at,full_text
0,Thu Nov 26 12:56:24 +0000 2020,"Het beroep van #apotheker, maar ook andere zorgberoepen zijn minder aantrekkelijk dan 2 jaar geleden. Boosdoener: #covid19. Gevaarlijke tendens, gezien het maatschappelijk belang van deze beroepen. \n@wbeke #zorg #health https://t.co/cdusYnc87a"
1,Thu Nov 26 12:56:21 +0000 2020,Dankjewel @maaikehartjes voor de sterke verbeelding van #covid19 #vaccins 💉 https://t.co/og2OeWBIsR
2,Thu Nov 26 12:56:15 +0000 2020,Een volledige evalutie over de werkwijze van het kabinet tijdens COVID-19 kunnen we met deze kennis ook wel afschrijven. https://t.co/KlXMHjshbs
3,Thu Nov 26 12:56:11 +0000 2020,"@hugodejonge @vierdaagsefeest @hugodejonge @vierdaagsefeest @St_DE4DAAGSE #Corona #coronapaspoort #vaccinatie : dit kan slimmer , veiliger en niet te hacken: https://t.co/olVBMEunYp"
4,Thu Nov 26 12:56:00 +0000 2020,"Onwaarschijnlijke reportage. Wij zijn LETTERLIJK de slechtste ter wereld, en toch blijven ondertussen vervangen ministers zich op de schouders kloppen en anderen blijven bewoon gewoon op post (Beke). \n\nGooi ze uit het raam. Het is het enigste dat nog kan.\n\nhttps://t.co/rBecJStt1J"


In [5]:
# Inspect datatypes
tweets.dtypes

created_at    object
full_text     object
dtype: object

In [6]:
# Convert created_at column to datetime
tweets['created_at'] = pd.to_datetime(tweets['created_at'])
tweets.dtypes

created_at    datetime64[ns, UTC]
full_text                  object
dtype: object

#### Calculating in the dataframe

1. First create an extra column to map the time difference since lockdown widening. Interpretation of the value:
    - Negative timedifference: the tweet was send before the widening of the measures
    - Positive timedifference: the tweet was send after the widening of the measures
2. Oldest message is from the 17th of November 20:59.
3. Most recent message is from 26th of November 12:56.

In [7]:
# First add a column with the date of the widening of the lockdown
tweets['lockdown_widening'] = '2020-11-19 00:00:01+00:00'

# Convert column to datetime
tweets['lockdown_widening'] = pd.to_datetime(tweets['lockdown_widening'])

# Check dtypes in dataframe
tweets.dtypes

# Create an extra column: time difference
tweets['timedelta'] = tweets['created_at'] - tweets['lockdown_widening']

# Inspect head of dataset
tweets.tail()

,created_at,full_text,lockdown_widening,timedelta
17779,2020-11-17 21:02:34+00:00,"Jaaa!!!! Malta komt op 1-0 tegen De Faröer eilanden. Je moet wat in zo een #lockdown periode op een trainingsavond. Die Pipo met zijn gekke schoenen wil dat we bewegen, maar volkssport nr 1 ligt voor 18 + al een half jaar stil #corona #persconferentie #dejonge",2020-11-19 00:00:01+00:00,-2 days +21:02:33
17780,2020-11-17 21:01:33+00:00,"https://t.co/UFvKwkheyy\nOok wij nemen tijdelijk een break ! \nDus muziek geen probleem, hier is DJ Luna 😉😅\nEen poging om u op te beuren. \nDank u om te blijven volhouden aan de maatregelen omtrent #Covid_19 #coronavirus @hoerenxxx @RedlightsBelgie @YourRedWish1",2020-11-19 00:00:01+00:00,-2 days +21:01:32
17781,2020-11-17 21:00:19+00:00,Voor jaaroverzicht terugkijken naar alle items in redactiesysteem. Valt nu op; hoe ontzettend snel er een eerste miljarden steunpakket klaar stond #corona #maart,2020-11-19 00:00:01+00:00,-2 days +21:00:18
17782,2020-11-17 21:00:01+00:00,@hugodejonge is rijkelijk laat met zijn 2e covid19 golf. HongKong was op 20 Oktober al bezig aan de 4e golf. Waarschijnlijk te druk met opkopen van boerengrond?😂🙈\nCoronavirus: How close is Hong Kong to a fourth wave of #COVID19?\n\nhttps://t.co/elfZBIBdbq https://t.co/l36JdBcpt4,2020-11-19 00:00:01+00:00,-2 days +21:00:00
17783,2020-11-17 20:59:19+00:00,Besef even dat onze hele maatschappij wordt ingericht met als doel &lt; 3 Covid-19 IC-opnames per dag. Alles daarboven is een probleem volgens @hugodejonge \nOnze zorg zelf is écht doodziek als dit de norm is die ervoor moet zorgen dat onze zorgsector niet overbelast raakt...,2020-11-19 00:00:01+00:00,-2 days +20:59:18


In [8]:
# Oldest message
tweets['created_at'].min()

Timestamp('2020-11-17 20:59:19+0000', tz='UTC')

In [9]:
# Most recent message
tweets['created_at'].max()

Timestamp('2020-11-26 12:56:24+0000', tz='UTC')

## Obtaining labeled Twitter messages
To be able to create a ML algorithm that predicts the sentiment of a message (negative or positive) there needs to be a labeled dataset, i.e. a dataset with two columns:
1. The text of a messages
2. The sentiment of the message (positive or negative).

Since I couldn't find a pre labeled dataset, I will create one myself.

#### Creating a sample and population dataset
*Disclaimer: Since the list with random numbers is modified each time this kernel runs, the sample dataset will differ. Therefore the code to save it to a csv file is preceded by a #. So it will not overwrite the csv files*.

1. Create a sample dataset using random numbers. Save it as a dataframe.
2. Save remaining data in a seperate dataframe.

Because the script generates new random numbers everytime it runs, I ask you to save the data in an Excel file and re upload it. We than work with the same set of text messages everytime. This makes it easier to make conclusions.

In [10]:
# Number of rows in the dataset (population)
population_size = len(tweets)
population_size

17784

In [11]:
# Create a list with 500 random integers
randomlist = random.sample(range(0, population_size), 500)

In [12]:
# Select rows corresponding to the 500 random chosen integers
tweets_sample = tweets.iloc[randomlist]
tweets_sample.head()

,created_at,full_text,lockdown_widening,timedelta
15014,2020-11-18 21:57:13+00:00,Volgens #Predikant Uitslag uit #Urk is #Corona en de maatschappelijke gevolgen geen thema in de kerkdienst. Alleen het woord van God is aan de orde.\nIn welke tijd leeft deze gemeenschap eigenlijk?,2020-11-19 00:00:01+00:00,-1 days +21:57:12
17712,2020-11-17 21:36:48+00:00,"@hankd79 @bslagter @DiederikSmit Maatregelen zijn er om overbelasting van de zorg te voorkomen. IFR van COVID-19 is NIET 0,2 procent",2020-11-19 00:00:01+00:00,-2 days +21:36:47
13334,2020-11-19 14:36:36+00:00,@edwinveldhuizen ⚡ Goirle ➚\n\n+12 sinds gisteren\n+518 sinds 1 okt\n+73 sinds 7 dagen (12 nov)\nWat meer is dan de +61 in de 7d ervoor\n\n23.839 inwoners maakt dat\n⚡ 306.2 / 100.000 / 7d\n\nDaarbij in de laatste 2 weken:\n+2 opnames\n\n[#COVID19NL 19 Nov],2020-11-19 00:00:01+00:00,0 days 14:36:35
5861,2020-11-23 13:28:05+00:00,@edwinveldhuizen 🟣 Drechterland ➘\n\n+5 sinds gisteren\n+363 sinds 1 okt\n+36 sinds 7 dagen (16 nov)\nWat minder is dan de +40 in de 7d ervoor\n\n19.801 inwoners maakt dat\n🟣 181.8 / 100.000 / 7d\n\n[#COVID19NL 23 Nov],2020-11-19 00:00:01+00:00,4 days 13:28:04
2277,2020-11-25 12:03:50+00:00,Helaas wordt dit opnieuw bevestigd: Het belang van handen wassen voor en na boodschappen onderstrepend! #COVID19 \n@VTMNIEUWS @vrtnws @be_gezondheid @ZorgVlaanderen \n\nFood Products as Potential Carriers of SARS-CoV-2 https://t.co/h8iNaDrIp5,2020-11-19 00:00:01+00:00,6 days 12:03:49


In [13]:
# Select rows not corresponding to the 500 random chosen integers
tweets_remaining = tweets.loc[~tweets.index.isin(randomlist)]
tweets_remaining.head()

,created_at,full_text,lockdown_widening,timedelta
0,2020-11-26 12:56:24+00:00,"Het beroep van #apotheker, maar ook andere zorgberoepen zijn minder aantrekkelijk dan 2 jaar geleden. Boosdoener: #covid19. Gevaarlijke tendens, gezien het maatschappelijk belang van deze beroepen. \n@wbeke #zorg #health https://t.co/cdusYnc87a",2020-11-19 00:00:01+00:00,7 days 12:56:23
1,2020-11-26 12:56:21+00:00,Dankjewel @maaikehartjes voor de sterke verbeelding van #covid19 #vaccins 💉 https://t.co/og2OeWBIsR,2020-11-19 00:00:01+00:00,7 days 12:56:20
2,2020-11-26 12:56:15+00:00,Een volledige evalutie over de werkwijze van het kabinet tijdens COVID-19 kunnen we met deze kennis ook wel afschrijven. https://t.co/KlXMHjshbs,2020-11-19 00:00:01+00:00,7 days 12:56:14
3,2020-11-26 12:56:11+00:00,"@hugodejonge @vierdaagsefeest @hugodejonge @vierdaagsefeest @St_DE4DAAGSE #Corona #coronapaspoort #vaccinatie : dit kan slimmer , veiliger en niet te hacken: https://t.co/olVBMEunYp",2020-11-19 00:00:01+00:00,7 days 12:56:10
4,2020-11-26 12:56:00+00:00,"Onwaarschijnlijke reportage. Wij zijn LETTERLIJK de slechtste ter wereld, en toch blijven ondertussen vervangen ministers zich op de schouders kloppen en anderen blijven bewoon gewoon op post (Beke). \n\nGooi ze uit het raam. Het is het enigste dat nog kan.\n\nhttps://t.co/rBecJStt1J",2020-11-19 00:00:01+00:00,7 days 12:55:59


### Second round of data cleaning
To be able to analyse or process the text data, it's important that the text is readable for a computer. This means removing special or unusefull charachters:
* Urls
* @user
* New line characters
* Emojis
* Upper cases
* Non-alphanumeric characters

Two remarks:
1. I choose not to remove the words followd by hashtags (only the hashtags itself), because they can hold valuable information about the sentiment.
2. I choose to not remove stopwords. Words like 'but' and 'not' (often considered stopwords) can hold valuable information about the sentiment.

In [14]:
# NLTK library provides lists of stopwords in multiple languages. They are also available in Dutch:
print(stopwords.words('dutch'))

# I choose not to use this list, because some words might influence the outcome of the sentiment analysis, like 'niet' and 'maar'.

['de', 'en', 'van', 'ik', 'te', 'dat', 'die', 'in', 'een', 'hij', 'het', 'niet', 'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 'niets', 'uw', 'iemand', 'geweest', 'andere']


In [15]:
# First remove urls & users
def cleaning(text):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub(r'', text)
    
    #Removing URLs with a regular expression
    user_pattern = re.compile(r'@[^\s]+')
    text = user_pattern.sub(r'', text)
    
    # Remove new line characters
    text = re.sub('\s+', ' ', text)
    
    # remove emojis
    text = re.sub(emoji.get_emoji_regexp(), r"", text)
        
    # remove upper cases
    text = text.lower()
    
    # remove non alphanumeric characters
    text = re.sub('\W', ' ', text)
    
    return text
    
# test if it works
text = tweets_remaining['full_text'].iloc[10]
cleaning(text)

'de procedure tijdelijke werkloosheid voor bouwvakarbeiders werd vereenvoudigd  tot 31 03 2021 hoef je niet in bezit te zijn van een controlekaart c3 2a  maar kan je een werkloosheidsaanvraag indienen  lees meer  '

In [16]:
# Map the function to the text column
tweets_sample['clean_text'] = tweets_sample['full_text'].map(cleaning)
tweets_sample.head()

<ipython-input-16-7b0d46e14f50>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_sample['clean_text'] = tweets_sample['full_text'].map(cleaning)


,created_at,full_text,lockdown_widening,timedelta,clean_text
15014,2020-11-18 21:57:13+00:00,Volgens #Predikant Uitslag uit #Urk is #Corona en de maatschappelijke gevolgen geen thema in de kerkdienst. Alleen het woord van God is aan de orde.\nIn welke tijd leeft deze gemeenschap eigenlijk?,2020-11-19 00:00:01+00:00,-1 days +21:57:12,volgens predikant uitslag uit urk is corona en de maatschappelijke gevolgen geen thema in de kerkdienst alleen het woord van god is aan de orde in welke tijd leeft deze gemeenschap eigenlijk
17712,2020-11-17 21:36:48+00:00,"@hankd79 @bslagter @DiederikSmit Maatregelen zijn er om overbelasting van de zorg te voorkomen. IFR van COVID-19 is NIET 0,2 procent",2020-11-19 00:00:01+00:00,-2 days +21:36:47,maatregelen zijn er om overbelasting van de zorg te voorkomen ifr van covid 19 is niet 0 2 procent
13334,2020-11-19 14:36:36+00:00,@edwinveldhuizen ⚡ Goirle ➚\n\n+12 sinds gisteren\n+518 sinds 1 okt\n+73 sinds 7 dagen (12 nov)\nWat meer is dan de +61 in de 7d ervoor\n\n23.839 inwoners maakt dat\n⚡ 306.2 / 100.000 / 7d\n\nDaarbij in de laatste 2 weken:\n+2 opnames\n\n[#COVID19NL 19 Nov],2020-11-19 00:00:01+00:00,0 days 14:36:35,goirle 12 sinds gisteren 518 sinds 1 okt 73 sinds 7 dagen 12 nov wat meer is dan de 61 in de 7d ervoor 23 839 inwoners maakt dat 306 2 100 000 7d daarbij in de laatste 2 weken 2 opnames covid19nl 19 nov
5861,2020-11-23 13:28:05+00:00,@edwinveldhuizen 🟣 Drechterland ➘\n\n+5 sinds gisteren\n+363 sinds 1 okt\n+36 sinds 7 dagen (16 nov)\nWat minder is dan de +40 in de 7d ervoor\n\n19.801 inwoners maakt dat\n🟣 181.8 / 100.000 / 7d\n\n[#COVID19NL 23 Nov],2020-11-19 00:00:01+00:00,4 days 13:28:04,drechterland 5 sinds gisteren 363 sinds 1 okt 36 sinds 7 dagen 16 nov wat minder is dan de 40 in de 7d ervoor 19 801 inwoners maakt dat 181 8 100 000 7d covid19nl 23 nov
2277,2020-11-25 12:03:50+00:00,Helaas wordt dit opnieuw bevestigd: Het belang van handen wassen voor en na boodschappen onderstrepend! #COVID19 \n@VTMNIEUWS @vrtnws @be_gezondheid @ZorgVlaanderen \n\nFood Products as Potential Carriers of SARS-CoV-2 https://t.co/h8iNaDrIp5,2020-11-19 00:00:01+00:00,6 days 12:03:49,helaas wordt dit opnieuw bevestigd het belang van handen wassen voor en na boodschappen onderstrepend covid19 food products as potential carriers of sars cov 2


In [17]:
# Same for remaining dataset
tweets_remaining['clean_text'] = tweets_remaining['full_text'].map(cleaning)
tweets_remaining.head()

# Save dataframe as csv file
tweets_remaining.to_csv("data\\twitter_remaining_cleaned.csv", sep=',', index=False)

<ipython-input-17-3d5a7ce6eca3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_remaining['clean_text'] = tweets_remaining['full_text'].map(cleaning)


#### Determine the sentiment of the sample dataset using Textblob
First create an extra column with the outcome of the Textblob sentiment analyzer. This results in a value that is a tuple (polarity, subjectivity):
* Polarity is a float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement.
* Subjectivity sentences generally refer to personal opinion, emotion or judgment whereas objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].

In [18]:
# Define a function that returns the sentiment of a string.
def sentiment(text):
    """input: string variable (text) you want to know the sentiment of
    output: sentiment in a tuple (polarity, subjectivity)
    """
    blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
    sentiment = blob.sentiment
    return sentiment

# test with example sentence, outcome should be (-0.1, 0.4)
text = "De kat wil wel vis eten maar geen poot nat maken."
sentiment(text)

(-0.1, 0.4)

In [19]:
# Map the function to the text column
tweets_sample['sentiment'] = tweets_sample['clean_text'].map(sentiment)
tweets_sample.head()

<ipython-input-19-b763bacdb7b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_sample['sentiment'] = tweets_sample['clean_text'].map(sentiment)


,created_at,full_text,lockdown_widening,timedelta,clean_text,sentiment
15014,2020-11-18 21:57:13+00:00,Volgens #Predikant Uitslag uit #Urk is #Corona en de maatschappelijke gevolgen geen thema in de kerkdienst. Alleen het woord van God is aan de orde.\nIn welke tijd leeft deze gemeenschap eigenlijk?,2020-11-19 00:00:01+00:00,-1 days +21:57:12,volgens predikant uitslag uit urk is corona en de maatschappelijke gevolgen geen thema in de kerkdienst alleen het woord van god is aan de orde in welke tijd leeft deze gemeenschap eigenlijk,"(0.0, 0.175)"
17712,2020-11-17 21:36:48+00:00,"@hankd79 @bslagter @DiederikSmit Maatregelen zijn er om overbelasting van de zorg te voorkomen. IFR van COVID-19 is NIET 0,2 procent",2020-11-19 00:00:01+00:00,-2 days +21:36:47,maatregelen zijn er om overbelasting van de zorg te voorkomen ifr van covid 19 is niet 0 2 procent,"(0.0, 0.0)"
13334,2020-11-19 14:36:36+00:00,@edwinveldhuizen ⚡ Goirle ➚\n\n+12 sinds gisteren\n+518 sinds 1 okt\n+73 sinds 7 dagen (12 nov)\nWat meer is dan de +61 in de 7d ervoor\n\n23.839 inwoners maakt dat\n⚡ 306.2 / 100.000 / 7d\n\nDaarbij in de laatste 2 weken:\n+2 opnames\n\n[#COVID19NL 19 Nov],2020-11-19 00:00:01+00:00,0 days 14:36:35,goirle 12 sinds gisteren 518 sinds 1 okt 73 sinds 7 dagen 12 nov wat meer is dan de 61 in de 7d ervoor 23 839 inwoners maakt dat 306 2 100 000 7d daarbij in de laatste 2 weken 2 opnames covid19nl 19 nov,"(-0.05, 0.2)"
5861,2020-11-23 13:28:05+00:00,@edwinveldhuizen 🟣 Drechterland ➘\n\n+5 sinds gisteren\n+363 sinds 1 okt\n+36 sinds 7 dagen (16 nov)\nWat minder is dan de +40 in de 7d ervoor\n\n19.801 inwoners maakt dat\n🟣 181.8 / 100.000 / 7d\n\n[#COVID19NL 23 Nov],2020-11-19 00:00:01+00:00,4 days 13:28:04,drechterland 5 sinds gisteren 363 sinds 1 okt 36 sinds 7 dagen 16 nov wat minder is dan de 40 in de 7d ervoor 19 801 inwoners maakt dat 181 8 100 000 7d covid19nl 23 nov,"(0.0, 0.0)"
2277,2020-11-25 12:03:50+00:00,Helaas wordt dit opnieuw bevestigd: Het belang van handen wassen voor en na boodschappen onderstrepend! #COVID19 \n@VTMNIEUWS @vrtnws @be_gezondheid @ZorgVlaanderen \n\nFood Products as Potential Carriers of SARS-CoV-2 https://t.co/h8iNaDrIp5,2020-11-19 00:00:01+00:00,6 days 12:03:49,helaas wordt dit opnieuw bevestigd het belang van handen wassen voor en na boodschappen onderstrepend covid19 food products as potential carriers of sars cov 2,"(0.0, 0.0)"


In [20]:
# Split the sentiment column into two columns: polarity and subjectivity
tweets_sample['sentiment'].tolist()                                                                                                                                                        
pd.DataFrame(tweets_sample['sentiment'].tolist(), index=tweets_sample.index)                                                                                                                                          
tweets_sample[['polarity', 'subjectivity']] = pd.DataFrame(tweets_sample['sentiment'].tolist(), index=tweets_sample.index)  
tweets_sample.head()

C:\Users\RianneRendering\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,created_at,full_text,lockdown_widening,timedelta,clean_text,sentiment,polarity,subjectivity
15014,2020-11-18 21:57:13+00:00,Volgens #Predikant Uitslag uit #Urk is #Corona en de maatschappelijke gevolgen geen thema in de kerkdienst. Alleen het woord van God is aan de orde.\nIn welke tijd leeft deze gemeenschap eigenlijk?,2020-11-19 00:00:01+00:00,-1 days +21:57:12,volgens predikant uitslag uit urk is corona en de maatschappelijke gevolgen geen thema in de kerkdienst alleen het woord van god is aan de orde in welke tijd leeft deze gemeenschap eigenlijk,"(0.0, 0.175)",0.00,0.175
17712,2020-11-17 21:36:48+00:00,"@hankd79 @bslagter @DiederikSmit Maatregelen zijn er om overbelasting van de zorg te voorkomen. IFR van COVID-19 is NIET 0,2 procent",2020-11-19 00:00:01+00:00,-2 days +21:36:47,maatregelen zijn er om overbelasting van de zorg te voorkomen ifr van covid 19 is niet 0 2 procent,"(0.0, 0.0)",0.00,0.000
13334,2020-11-19 14:36:36+00:00,@edwinveldhuizen ⚡ Goirle ➚\n\n+12 sinds gisteren\n+518 sinds 1 okt\n+73 sinds 7 dagen (12 nov)\nWat meer is dan de +61 in de 7d ervoor\n\n23.839 inwoners maakt dat\n⚡ 306.2 / 100.000 / 7d\n\nDaarbij in de laatste 2 weken:\n+2 opnames\n\n[#COVID19NL 19 Nov],2020-11-19 00:00:01+00:00,0 days 14:36:35,goirle 12 sinds gisteren 518 sinds 1 okt 73 sinds 7 dagen 12 nov wat meer is dan de 61 in de 7d ervoor 23 839 inwoners maakt dat 306 2 100 000 7d daarbij in de laatste 2 weken 2 opnames covid19nl 19 nov,"(-0.05, 0.2)",-0.05,0.200
5861,2020-11-23 13:28:05+00:00,@edwinveldhuizen 🟣 Drechterland ➘\n\n+5 sinds gisteren\n+363 sinds 1 okt\n+36 sinds 7 dagen (16 nov)\nWat minder is dan de +40 in de 7d ervoor\n\n19.801 inwoners maakt dat\n🟣 181.8 / 100.000 / 7d\n\n[#COVID19NL 23 Nov],2020-11-19 00:00:01+00:00,4 days 13:28:04,drechterland 5 sinds gisteren 363 sinds 1 okt 36 sinds 7 dagen 16 nov wat minder is dan de 40 in de 7d ervoor 19 801 inwoners maakt dat 181 8 100 000 7d covid19nl 23 nov,"(0.0, 0.0)",0.00,0.000
2277,2020-11-25 12:03:50+00:00,Helaas wordt dit opnieuw bevestigd: Het belang van handen wassen voor en na boodschappen onderstrepend! #COVID19 \n@VTMNIEUWS @vrtnws @be_gezondheid @ZorgVlaanderen \n\nFood Products as Potential Carriers of SARS-CoV-2 https://t.co/h8iNaDrIp5,2020-11-19 00:00:01+00:00,6 days 12:03:49,helaas wordt dit opnieuw bevestigd het belang van handen wassen voor en na boodschappen onderstrepend covid19 food products as potential carriers of sars cov 2,"(0.0, 0.0)",0.00,0.000


#### Based on TextBlob, categorize sentiment as positive, negative or neutral

In [21]:
# Define a function to determine the sentiment category
def sentiment_cat(polarity):
    """input: float, based on the output of TextBlob polarity measure, you want to categorize
    output: sentiment category, either positive, negative or neutral
    """
    polarity = float(polarity)
    if polarity > 0:
        return 'positive'
    
    if polarity < 0:
        return 'negative'
    
    if polarity == 0:
        return 'neutral'
    
    else:
        return 'error'
    
# test with example (-0.1 from previous example) should return negative
polarity = "-0.1"
sentiment_cat(polarity)

'negative'

In [22]:
# Map the function to the polarity column
tweets_sample['sentiment_cat'] = tweets_sample['polarity'].map(sentiment_cat)
tweets_sample.head()

<ipython-input-22-e4e83591df82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_sample['sentiment_cat'] = tweets_sample['polarity'].map(sentiment_cat)


,created_at,full_text,lockdown_widening,timedelta,clean_text,sentiment,polarity,subjectivity,sentiment_cat
15014,2020-11-18 21:57:13+00:00,Volgens #Predikant Uitslag uit #Urk is #Corona en de maatschappelijke gevolgen geen thema in de kerkdienst. Alleen het woord van God is aan de orde.\nIn welke tijd leeft deze gemeenschap eigenlijk?,2020-11-19 00:00:01+00:00,-1 days +21:57:12,volgens predikant uitslag uit urk is corona en de maatschappelijke gevolgen geen thema in de kerkdienst alleen het woord van god is aan de orde in welke tijd leeft deze gemeenschap eigenlijk,"(0.0, 0.175)",0.00,0.175,neutral
17712,2020-11-17 21:36:48+00:00,"@hankd79 @bslagter @DiederikSmit Maatregelen zijn er om overbelasting van de zorg te voorkomen. IFR van COVID-19 is NIET 0,2 procent",2020-11-19 00:00:01+00:00,-2 days +21:36:47,maatregelen zijn er om overbelasting van de zorg te voorkomen ifr van covid 19 is niet 0 2 procent,"(0.0, 0.0)",0.00,0.000,neutral
13334,2020-11-19 14:36:36+00:00,@edwinveldhuizen ⚡ Goirle ➚\n\n+12 sinds gisteren\n+518 sinds 1 okt\n+73 sinds 7 dagen (12 nov)\nWat meer is dan de +61 in de 7d ervoor\n\n23.839 inwoners maakt dat\n⚡ 306.2 / 100.000 / 7d\n\nDaarbij in de laatste 2 weken:\n+2 opnames\n\n[#COVID19NL 19 Nov],2020-11-19 00:00:01+00:00,0 days 14:36:35,goirle 12 sinds gisteren 518 sinds 1 okt 73 sinds 7 dagen 12 nov wat meer is dan de 61 in de 7d ervoor 23 839 inwoners maakt dat 306 2 100 000 7d daarbij in de laatste 2 weken 2 opnames covid19nl 19 nov,"(-0.05, 0.2)",-0.05,0.200,negative
5861,2020-11-23 13:28:05+00:00,@edwinveldhuizen 🟣 Drechterland ➘\n\n+5 sinds gisteren\n+363 sinds 1 okt\n+36 sinds 7 dagen (16 nov)\nWat minder is dan de +40 in de 7d ervoor\n\n19.801 inwoners maakt dat\n🟣 181.8 / 100.000 / 7d\n\n[#COVID19NL 23 Nov],2020-11-19 00:00:01+00:00,4 days 13:28:04,drechterland 5 sinds gisteren 363 sinds 1 okt 36 sinds 7 dagen 16 nov wat minder is dan de 40 in de 7d ervoor 19 801 inwoners maakt dat 181 8 100 000 7d covid19nl 23 nov,"(0.0, 0.0)",0.00,0.000,neutral
2277,2020-11-25 12:03:50+00:00,Helaas wordt dit opnieuw bevestigd: Het belang van handen wassen voor en na boodschappen onderstrepend! #COVID19 \n@VTMNIEUWS @vrtnws @be_gezondheid @ZorgVlaanderen \n\nFood Products as Potential Carriers of SARS-CoV-2 https://t.co/h8iNaDrIp5,2020-11-19 00:00:01+00:00,6 days 12:03:49,helaas wordt dit opnieuw bevestigd het belang van handen wassen voor en na boodschappen onderstrepend covid19 food products as potential carriers of sars cov 2,"(0.0, 0.0)",0.00,0.000,neutral


#### Save output as Excel file. This output will be manually judged outside Python

In [23]:
# Remove unnecessary columns. Remaining columns: text and sentiment_cat
tweets_sample_cat = tweets_sample[['clean_text', 'sentiment_cat']]

In [24]:
# Save dataframe as excel file
tweets_sample_cat.to_csv("data\\twitter_sample_sentiment_cleaned.csv", sep=',', index=False)

# To do:
2. Stemming/Lemmatizing